In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import sys, os

## 5.4.1 곱셉 계층

In [2]:
class MulLayer :
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self,x,y):
        self.x = x
        self.y = y
        out = x*y
        return out
    
    def backward(self,dout):
        dx = dout *self.y
        dy = dout*self.x
        return dx, dy

In [4]:
apple =100
apple_num = 2
tax =1.1

#계층설정
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple,apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [6]:
#역전파(미분)
dprice =1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple)
print(dapple_num)
print(dtax)

2.2
110.00000000000001
200


## 5.4.2 덧셈계층

In [8]:
class AddLayer:
    def __init__(self):
        pass
    # 덧셈 계층에서는 초기화가 필요없다. 
    
    def forward(self, x, y):
        out = x+y
        return x+y
    
    def backward(self,dout):
        dx = dout *1
        dy = dout*1
        return dx, dy

In [13]:
#그림 5.17 (p.163)

apple=100
apple_num = 2
orange =150
orange_num = 3
tax = 1.1

#계층 설정
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price  = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price  = add_apple_orange_layer.backward(dall_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_apple_layer.backward(dorange_price)

In [16]:
print(price)
print(dapple_num, dapple)
print(dorange, dorange_num)
print(dtax)

715.0000000000001
110.00000000000001 2.2
2.2 110.00000000000001
650


# 5.5 활성화 계층 구현하기(ReLU, sigmoid)

## 5.5.1 ReLU 계층

In [18]:
class ReLU:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout
        return dx
    
x = np.array([[10., -0.5], [-2.0, 3.0]])
print(x)

mask = (x<=0)
print(mask)

[[10.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


## 5.5.2 Sigmoid 계층

In [19]:
class sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1+np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 -self.out) *self.out
        return dx

# 5.6 Affine, Softmax 계층 구현
## 5.6.1 Affine 계층 
 - 신경망의 순전파에서 수행하는 행렬의 곱은 기하학에서 Affine transformation 이라고 한다. > affine trans. 를하는 처리를 affine 계층이라고 한다.

In [21]:
X= np.random.rand(2)
W = np.random.rand(2,3)
B = np.random.rand(3)

print(X.shape)
print(W.shape)
print(B.shape)

(2,)
(2, 3)
(3,)


## 5.6.2 배치용 Affine 계층
- N개의 데이터를 묶어 순전파 하는경우. 

In [24]:
X_dot_W = np.array([[0,0,0], [10, 10, 10]])
B = np.array([1,2,3])

print(X_dot_W)
print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [27]:
dY = np.array([[1,2,3], [4,5,6]])
print(dY)

dB = np.sum(dY, axis = 0)
print(dB)


[[1 2 3]
 [4 5 6]]
[5 7 9]


In [29]:
class Affine:
    def __init(self, W, b):
        self.W =W
        self,b =b
        self.x= None
        self.dW = None
        slef.df = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return  out
    
    def backward(self, dout):
        dx = np.dot(dout, W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.dot(dout, axis=0)
        return dx

## 5.6.3 Softmax with Loss 계층 
- 신경망에서 수행하는 작업은 학습, 추론. 추론 시 일반적으로 softmax계층을 사용하지 않는다. 
- 신경망의 추론은 마지막 affine 계층의 출력을 인식결과로 이용.
- 정규화하지 않은 출력결과를 점수라고 한다. 
- 신경망 추론에서 답을 1개만 내는 경우에는 softmax 계층 불필요. 하지만 신경망 학습에는 softmax가 필요하다.

In [30]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t=t
        self.y=softmax(x)
        self.loss = cross_entropy_error(self.y, self,t)
        return self.loss
    
    def backward(self, dout =1):
        batch_size = self.t.shape[0]
        dx = (self.y- self.t) /batch_size
        return dx

# 5.7 오차역전파법 구현하기
- 전제: 신경망에는 적응가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈현 데이터에 적응하도록 조정하는 과정을  '학습'이라고 한다.
- 1단계: 미니배치 >> 훈련 데이터중 일부를 무작위로 가져온다. 선별한 데이터를 미니배치라고 하며, 이 미니배치의 손실 함수 값을 줄이는 것이 목표
- 2단계: 기울기 산출 >> 미니배치의 손실함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실함수의 값을 가장 작게. 
- 3단계: 가중치 매개변수를 기울기 방향으로 아주 조금 갱신
- 4단계: 1~3단계를 반복한다.

#### 2 layer로 구현을 할것이다.
#### 인스턴스 변수: params, layers, lastLayer(softmax)
#### 클래스의 매서드: __init__, predict, loss, accuracy, numerical_gradient, gradient

In [47]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.rand(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        
        self.params['W2'] = weight_init_std * np.random.rand(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        #계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['ReLu1'] =  Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
           
    def loss(self, x, t):
        y= self.predict(x)
        return self.lastLayer.forward(y,t)
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y =np.argmax(y, axis=1)
        if t.ndim != 1: t = np.argmax(t, axis=1)
        accuracy = np.sum(y==t)/float(x.shape[0])
        return accuracy

    def numerical_gradient(self,x,t):
        loss_W = lambda W: self.loss(x,t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self,x,t):
        # 순전파
        self.loss(x,t)
        
        #역전파
        dout=1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과 저장. 
        grads= {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads
        
    
        

### 신경망 계층을 OderedDict에 보관하는것이 중요하다.
 - OrderedDict는 순서가 있는 딕셔너리이다. 딕셔너리에 추가한 순서를 기억하게 하는것. >> 순전파 때는 추가한 순서대로 각 계층의 forward를 호출하면 된다.
 - 마찬가지로 역전파때는 계층을 반대 순서로 호출하기만 하면 된다. 
 
### 신경망의 구성요들을 계층으로 모듈화 하여 구현하는 것은 아주 좋다. 
 - 더 깊은 신경망을 만들때 단순히 필요한 만큼의 계층을 추가하면 되기 때문이다. 

In [48]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist



(x_train,t_train), (x_test, t_Test) = load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key] ))
    print(key + ':' +str(diff))


W1:2.2486547439350546e-10
b1:1.0805058431935956e-09
W2:7.21377318080585e-08
b2:1.401186825525369e-07


## 5.7.4 오차역전파법을 사용한 학습 구현하기

In [53]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test,t_test) = load_mnist(normalize= True, one_hot_label = True)
network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size/ batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -=  learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)
    
        

0.09871666666666666 0.098
0.9009833333333334 0.9033
0.9191666666666667 0.9211
0.9330166666666667 0.9313
0.9430166666666666 0.9403
0.9494 0.9463
0.9564 0.9528
0.9599 0.9568
0.9628166666666667 0.96
0.9658333333333333 0.9631
0.9679166666666666 0.9638
0.9703833333333334 0.9648
0.9724666666666667 0.9661
0.9740166666666666 0.965
0.9756833333333333 0.968
0.9769666666666666 0.9683
0.9783333333333334 0.9682
